In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env, get_openai_model_name, get_judge_llm_model_name, get_openai_api_key
load_env()

import os
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

from pprint import pprint
import json

/Users/sanjujoseph/Workspace/GenAI/crewai/deeplearning.ai/practical_ai_agents/agentic_sales_pipeline/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/sanjujoseph/Workspace/GenAI/crewai/deeplearning.ai/practical_ai_agents/agentic_sales_pipeline/.venv/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @va

In [2]:
openai_model_name = get_openai_model_name()
print(f"openai_model_name: {openai_model_name}")
os.environ['OPENAI_MODEL_NAME'] = openai_model_name

openai_model_name: gpt-4o-mini


In [3]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# print(f"Configs List: {configs}")

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

In [4]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

In [5]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

In [6]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

In [7]:
from crewai import Flow
from crewai.flow.flow import listen, start


class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
            {
                "lead_data": {
                    "name": "Sanju Joseph",
                    "job_title": "Sr. Director, IT",
                    "company": "Western Alliance Bank",
                    "email": "antony.sanjujosepj@westernalliancebank.com",
                    "use_case": "Work with business to optimize business operations."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        print(f"Leads from the leads database: {leads}")
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        print(f"store_leads_score: Scores obtained: {scores}")
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        print(f"Email Crew Kick Off Complete. Emails: {emails}")
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: fb456591-0748-4ad0-9b4a-cf2f8721e770                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [8]:
flow.plot("simple_crewai_flow")
from IPython.display import IFrame

IFrame(src='./simple_crewai_flow.html', width='150%', height=600)

Plot saved as simple_crewai_flow.html


In [9]:
#!pip install nest-asyncio

In [10]:
import nest_asyncio
nest_asyncio.apply()

# # Now you can use asyncio.run() even in a running event loop
# emails = await flow.kickoff()  # Make sure this line is inside an async function

emails = flow.kickoff()

🌊 Flow: SalesPipeline
    ID: fb456591-0748-4ad0-9b4a-cf2f8721e770
└── 🧠 Starting Flow...

 Flow started with ID: fb456591-0748-4ad0-9b4a-cf2f8721e770


🌊 Flow: SalesPipeline
    ID: fb456591-0748-4ad0-9b4a-cf2f8721e770
├── 🧠 Starting Flow...
└── 🔄 Running: fetch_leads

🌊 Flow: SalesPipeline
    ID: fb456591-0748-4ad0-9b4a-cf2f8721e770
├── Flow Method Step
└── ✅ Completed: fetch_leads

🌊 Flow: SalesPipeline
    ID: fb456591-0748-4ad0-9b4a-cf2f8721e770
├── Flow Method Step
├── ✅ Completed: fetch_leads
└── 🔄 Running: score_leads

...

🌊 Flow: SalesPipeline
    ID: fb456591-0748-4ad0-9b4a-cf2f8721e770
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
└── ✅ Completed: write_email

🌊 Flow: SalesPipeline
    ID: fb456591-0748-4ad0-9b4a-cf2f8721e770
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── 🔄 Running: send_email

🌊 Flow: SalesPipeline
    ID: fb456591-0748-4ad0-9b4a-cf2f8721e770
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

✅ Flow Finished: SalesPipeline
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: SalesPipeline                                                                                            │
│  ID: fb456591-0748-4ad0-9b4a-cf2f8721e770                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Usage Metrics and Costs

In [11]:
print(f"PRINTING FLOW STATE - SCORE_CREW_RESULTS:\n {flow.state["score_crews_results"]}")

PRINTING FLOW STATE - SCORE_CREW_RESULTS:
 [CrewOutput(raw='{\n  "personal_info": {\n    "name": "João Moura",\n    "job_title": "Director of Engineering",\n    "role_relevance": 9,\n    "professional_background": "João has extensive experience in engineering management and has been involved in projects focusing on AI technologies for data enrichment."\n  },\n  "company_info": {\n    "company_name": "Clearbit",\n    "industry": "Marketing Data Technology",\n    "company_size": 150,\n    "revenue": 31.5,\n    "market_presence": 8\n  },\n  "lead_score": {\n    "score": 83,\n    "scoring_criteria": [\n      "Role Relevance: 9/10 - João\'s role as Director of Engineering aligns closely with our target ICP.",\n      "Company Size: 150 employees - This falls within the enterprise category that we target.",\n      "Market Presence: 8/10 - Clearbit has a strong presence and reputation in the marketing data technology space.",\n      "Cultural Fit: 8/10 - Their focus on customer-centricity and 

In [12]:
import pandas as pd
from pprint import pprint

# pprint (flow.state["score_crews_results"])
# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# # # Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# # # Display the DataFrame
df_usage_metrics

Total costs: $0.0041


/var/folders/lw/m3x_nm_552sdfjh1wg2jjphr0000gn/T/ipykernel_89474/3013968659.py:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,27386,25255,13696,2131,10


In [13]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0002


/var/folders/lw/m3x_nm_552sdfjh1wg2jjphr0000gn/T/ipykernel_89474/99626748.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,1272,1038,0,234,2


In [14]:
## Inspecting Results

In [22]:
scores = flow.state["score_crews_results"]

print("*****************************")
print(f"Scores Count: {len(scores)}")
for score in scores:
    print("*****************************")
    print(f"Score: {score}")


*****************************
Scores Count: 2
*****************************
Score: personal_info=LeadPersonalInfo(name='João Moura', job_title='Director of Engineering', role_relevance=9, professional_background='João has extensive experience in engineering management and has been involved in projects focusing on AI technologies for data enrichment.') company_info=CompanyInfo(company_name='Clearbit', industry='Marketing Data Technology', company_size=150, revenue=31.5, market_presence=8) lead_score=LeadScore(score=83, scoring_criteria=["Role Relevance: 9/10 - João's role as Director of Engineering aligns closely with our target ICP.", 'Company Size: 150 employees - This falls within the enterprise category that we target.', 'Market Presence: 8/10 - Clearbit has a strong presence and reputation in the marketing data technology space.', 'Cultural Fit: 8/10 - Their focus on customer-centricity and innovation matches our values.'], validation_notes='The score was validated against market p

In [23]:
import pandas as pd
from IPython.display import display, HTML

for score in scores:

    # lead_scoring_result = scores[0].pydantic
    lead_scoring_result = score.pydantic
    
    # Create a dictionary with the nested structure flattened
    data = {
        'Name': lead_scoring_result.personal_info.name,
        'Job Title': lead_scoring_result.personal_info.job_title,
        'Role Relevance': lead_scoring_result.personal_info.role_relevance,
        'Professional Background': lead_scoring_result.personal_info.professional_background,
        'Company Name': lead_scoring_result.company_info.company_name,
        'Industry': lead_scoring_result.company_info.industry,
        'Company Size': lead_scoring_result.company_info.company_size,
        'Revenue': lead_scoring_result.company_info.revenue,
        'Market Presence': lead_scoring_result.company_info.market_presence,
        'Lead Score': lead_scoring_result.lead_score.score,
        'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
        'Validation Notes': lead_scoring_result.lead_score.validation_notes
    }
    
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])
    
    # Reset the index to turn the original column names into a regular column
    df = df.reset_index()
    
    # Rename the index column to 'Attribute'
    df = df.rename(columns={'index': 'Attribute'})
    
    # Create HTML table with bold attributes and left-aligned values
    html_table = df.style.set_properties(**{'text-align': 'left'}) \
                         .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                         .hide(axis='index') \
                         .to_html()
    
    # Display the styled HTML table
    display(HTML(html_table))

Attribute,Value
Name,João Moura
Job Title,Director of Engineering
Role Relevance,9
Professional Background,João has extensive experience in engineering management and has been involved in projects focusing on AI technologies for data enrichment.
Company Name,Clearbit
Industry,Marketing Data Technology
Company Size,150
Revenue,31.500000
Market Presence,8
Lead Score,83


Attribute,Value
Name,Sanju Joseph
Job Title,"Sr. Director, IT"
Role Relevance,8
Professional Background,"Sr. Director in IT, focusing on optimizing business operations."
Company Name,Western Alliance Bank
Industry,Banking
Company Size,3260
Revenue,4316.000000
Market Presence,8
Lead Score,80


In [17]:
## Results

In [25]:
import textwrap

print("*****************************")
print(f"Emails Count: {len(emails)}")
for email in emails:
    print("*****************************")
    print(f"Email:\n {email}")
    
for email in emails:
    print("*****************************")
    # result_text = emails[0].raw
    result_text = email.raw
    wrapped_text = textwrap.fill(result_text, width=80)
    print(wrapped_text)


*****************************
Emails Count: 2
*****************************
Email:
 CrewAI’s Multi-Agent Orchestration Platform can transform Clearbit’s operational efficiency. By leveraging our platform, you can harness the power of AI agents tailored to elevate your marketing data technology. 

Ready to see how we can drive your automation strategy forward? **[Schedule a quick demo](#)** for this week. 

Act now and discover the impact of advanced automation on your workflows. **[Engage with us today](#)** to unlock innovation at Clearbit!
*****************************
Email:
 Sanju, optimizing your business operations at Western Alliance Bank is crucial, and CrewAI can help you achieve that effectively. Our Multi-Agent Orchestration Platform empowers enterprise-level clients by orchestrating AI Agents, streamlining your operations, and enhancing productivity instantly.

Imagine the efficiency gains and improved service delivery you can achieve. Let’s not wait to unlock this potentia